Generating Data

In [27]:
!pip install keras==2.10.0
!pip install tensorflow==2.10.0
!pip install twilio
!pip install opencv-python

     |████████████████████████████████| 60.9 MB 11.2 MB/s eta 0:00:01


In [10]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='O1W0QZh_MQESFYScengeecC0-JhiRZdlZ256g0aXQKaZ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'firedetector-donotdelete-pr-mmacx1ylnzryze'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [11]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    './Dataset/train_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    './Dataset/test_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 436 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


Building Model

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc']
)

model.summary()

2022-11-14 11:24:23.284555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-14 11:24:23.284604: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

Training and saving model

In [14]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)
model.save('forest1.h5')

Epoch 1/10
28/28 [==============================] - 34s 1s/step - loss: 0.5497 - acc: 0.7271 - val_loss: 0.3274 - val_acc: 0.8430
Epoch 2/10
28/28 [==============================] - 33s 1s/step - loss: 0.3685 - acc: 0.8601 - val_loss: 0.2665 - val_acc: 0.8678
Epoch 3/10
28/28 [==============================] - 33s 1s/step - loss: 0.3707 - acc: 0.8716 - val_loss: 0.0875 - val_acc: 0.9917
Epoch 4/10
28/28 [==============================] - 32s 1s/step - loss: 0.3175 - acc: 0.8761 - val_loss: 0.0768 - val_acc: 0.9917
Epoch 5/10
28/28 [==============================] - 32s 1s/step - loss: 0.2448 - acc: 0.8876 - val_loss: 0.0637 - val_acc: 0.9835
Epoch 6/10
28/28 [==============================] - 33s 1s/step - loss: 0.2396 - acc: 0.8991 - val_loss: 0.0781 - val_acc: 0.9669
Epoch 7/10
28/28 [==============================] - 33s 1s/step - loss: 0.3013 - acc: 0.8784 - val_loss: 0.0654 - val_acc: 0.9504
Epoch 8/10
28/28 [==============================] - 33s 1s/step - loss: 0.1921 - acc: 0.92

Fire Detection

In [15]:
!tar -zcvf fire_detector.tgz forest1.h5

forest1.h5


In [16]:
ls

Dataset/  fire_detector.tgz  forest1.h5


In [17]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 2.8 MB/s eta 0:00:01


In [18]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"zlDmR91GX7fNWn0H2eJiGXkOq22EVA6eOE0_sLFiLiRz"
}
client = APIClient(wml_credentials)

In [19]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [20]:
space_uid = guid_from_space_name(client, 'fire_detector')
print('Space UID = '+space_uid)

Space UID = c3c6fe4f-8eff-4aaa-998d-7ea014a78333


In [21]:
client.set.default_space(space_uid)

'SUCCESS'

In [22]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [23]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [24]:
model_details = client.repository.store_model(model='fire_detector.tgz', meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [25]:
model_id

'34bccf8f-2323-4de2-bb94-5a70ef74772a'

In [30]:
import cv2
import numpy as np
from keras.utils import load_img, img_to_array
from keras.models import load_model
from twilio.rest import Client

model = load_model(r'forest1.h5')
name = ['forest', 'with fire']


img = load_img(r'./Dataset/test_set/with fire/19464620_401.jpg', target_size = (150,150))
x = img_to_array(img)
x = np.expand_dims(x,axis = 0)
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)
p = pred[0][0].astype(int)
print(pred)
    
cv2.putText(frame, "predicted class ="+str(name[p]), (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),1)
    
pred = model.predict(x)
if pred[0]==1:
    account_sid = 'AC03faa7b579ff4212e3a310a93e553e1a'
    auth_token = 'c6e2eaada8fdabad91c463189525cf37'
    client = Client(account_sid, auth_token)
    message = client.messages \
        .create(
            body = 'Forest Fire is detected, stay alert',
            from_='+12182559719',
            to='+919600909256'
        )
    print(message.sid)
    print('Fire Detected')
    print('SMS sent!')
        
else:
    print("No Danger")

    


1/1 [==============================] - 0s 83ms/step
[[1.]]
1/1 [==============================] - 0s 36ms/step
SM60bdc855eb0776be058df205307d4566
Fire Detected
SMS sent!


In [5]:
pwd

'/home/wsuser/work'